In [1]:
import os
import random
from collections import Counter

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = '{:.4f}'.format

In [2]:
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV, LogisticRegression

In [3]:
def confusion_matrix_report(true, pred):
    [tn, fp, fn, tp] = confusion_matrix(true, pred).ravel()
    precision = tp/(fp + tp)
    recall = tp/(fn + tp)
    print(f"\t\tT\tF")
    print(f"\t1 [{tp:5d}, {fn:5d}]")
    print(f"\t0 [{fp:5d}, {tn:5d}]")
    print(f"accuracy : \t{ (tp+tn)/(tn + fp + fn + tp) :2.5f}")
    print(f"precision : \t{precision:2.5f}")
    print(f"recall : \t{recall:2.5f}")
    print(f"f1score : \t{(2*recall*precision)/(recall+precision):2.5f}")
    
    
def shuffle(x, y):
    idx = list(range(y_resample.shape[0]))
    random.shuffle(idx)
    return x[idx], y[idx]

In [4]:
dataset = pd.read_csv(os.path.join("/home/gruds/projects/chan/pico/dataset/credit_card","preprocessed.csv"), index_col = 0)

In [5]:
columns = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
           'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
           'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Time',
           'Class']
dataset = dataset[columns]

In [6]:
reduce_size = False
normalize = True

In [7]:
if reduce_size:
    columns = dataset.columns
    x = dataset.drop("Class",axis =1).values
    y = dataset.Class.values

    rus = RandomUnderSampler(random_state=42, sampling_strategy=0.01)
    x , y = rus.fit_resample(x, y)
    reduced = np.concatenate([x, y.reshape((-1,1))], axis =1)

    del x, y
    dataset = pd.DataFrame(data=reduced, columns=columns)

In [8]:
if normalize:
    is_fraud = dataset.Class.values.copy()

    scaler = StandardScaler() # RobustScaler # MinMaxScaler # StandardScaler

    transformed = dataset.drop("Class",axis =1)
    columns = transformed.columns
    transformed = scaler.fit_transform(transformed)

    del dataset
    dataset = pd.DataFrame(data=transformed, columns=columns)
    dataset = dataset.assign(Class = is_fraud)

In [9]:
x = dataset.drop('Class', axis =1)
y = dataset.Class

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# normal

In [11]:
Counter(y_train)

Counter({0: 227430, 1: 393})

In [12]:
logistic_regression = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True
                                        , intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100
                                        , multi_class='ovr', verbose=2, warm_start=False, n_jobs=6
                        )
logistic_regression.fit(x_train, y_train)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    9.4s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=6, penalty='l2', random_state=None,
                   solver='lbfgs', tol=0.0001, verbose=2, warm_start=False)

In [13]:
y_pred = logistic_regression.predict(x_test)

In [14]:
confusion_matrix_report(y_pred, y_test)

		T	F
	1 [   66,    11]
	0 [   29, 56850]
accuracy : 	0.99930
precision : 	0.69474
recall : 	0.85714
f1score : 	0.76744


# random oversampling

In [15]:
from imblearn.over_sampling import RandomOverSampler

In [16]:
ros = RandomOverSampler(sampling_strategy=0.01, random_state=42)
x_resample, y_resample = ros.fit_resample(x_train, y_train)

In [17]:
Counter(y_resample)

Counter({0: 227430, 1: 2274})

In [18]:
logistic_regression = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True
                                        , intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100
                                        , multi_class='ovr', verbose=2, warm_start=False, n_jobs=6
                        )
logistic_regression.fit(x_resample, y_resample)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    7.7s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=6, penalty='l2', random_state=None,
                   solver='lbfgs', tol=0.0001, verbose=2, warm_start=False)

In [19]:
y_pred = logistic_regression.predict(x_test)

In [20]:
confusion_matrix_report(y_pred, y_test)

		T	F
	1 [   79,    16]
	0 [   16, 56845]
accuracy : 	0.99944
precision : 	0.83158
recall : 	0.83158
f1score : 	0.83158


# variational oversampling

In [21]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [22]:
Counter(y_train)

Counter({0: 227430, 1: 393})

In [23]:
from vos import VariationalOversampler
variational_oversampler = VariationalOversampler(8)

tensorflow version 2.0.0


In [24]:
variational_oversampler.fit(x_train, y_train, 1)

Train on 314 samples, validate on 79 samples
Epoch 1/1000
314/314 [==============================] - 1s 2ms/sample - loss: 319120.0007 - val_loss: 44.8747
Epoch 2/1000
314/314 [==============================] - 0s 112us/sample - loss: 94.1769 - val_loss: 53.8107
Epoch 3/1000
314/314 [==============================] - 0s 125us/sample - loss: 96.0346 - val_loss: 35.6919
Epoch 4/1000
314/314 [==============================] - 0s 138us/sample - loss: 64.0151 - val_loss: 22.3542
Epoch 5/1000
314/314 [==============================] - 0s 133us/sample - loss: 53.1122 - val_loss: 21.0031
Epoch 6/1000
314/314 [==============================] - 0s 125us/sample - loss: 39.6669 - val_loss: 16.9379
Epoch 7/1000
314/314 [==============================] - 0s 128us/sample - loss: 47.9347 - val_loss: 16.5947
Epoch 8/1000
314/314 [==============================] - 0s 139us/sample - loss: 27.1041 - val_loss: 14.9781
Epoch 9/1000
314/314 [==============================] - 0s 118us/sample - loss: 25.0056 -

In [25]:
n_resample = int(Counter(y_train)[0] * 0.04 - Counter(y_train)[1])

In [26]:
x_gen = variational_oversampler.decoder.predict(np.random.normal(size = (n_resample, variational_oversampler.latent_dim)), verbose=1)

8704/8704 [==============================] - 0s 39us/sample


In [27]:
x_resample =  np.concatenate([x_train, x_gen], axis =0)
y_resample = np.concatenate([y_train, np.full((n_resample,) , 1)], axis =0)

In [28]:
x_resample, y_resample = shuffle(x_resample, y_resample)

In [29]:
logistic_regression = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True
                                        , intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100
                                        , multi_class='ovr', verbose=2, warm_start=False, n_jobs=6
                        )
logistic_regression.fit(x_resample, y_resample)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    6.2s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=6, penalty='l2', random_state=None,
                   solver='lbfgs', tol=0.0001, verbose=2, warm_start=False)

In [30]:
y_pred = logistic_regression.predict(x_test)

In [31]:
confusion_matrix_report(y_pred, y_test)

		T	F
	1 [   80,    47]
	0 [   15, 56814]
accuracy : 	0.99891
precision : 	0.84211
recall : 	0.62992
f1score : 	0.72072
